# Exporting Interpolating Object for Fitting Formula

## Import Packages

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import time

In [ ]:
from looti import dictlearn as dcl
from looti import datahandle as dhl
from looti import PlottingModule as pm

from looti import tools as too
from looti import PlottingModule as pm
from looti import interpolatingObject as ito

In [ ]:
import pickle
import joblib

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# Import Data

### Specify Paths

The DataFrames of the *extented model* and the *reference model* should be stored inside the same `data_folder`. The results of the experiments(plots, interpolation functions) are stored inside `the results_folder`

In [ ]:
data_folder = '../../SimulationData/CDE_fitting_formulae/'
results_folder = './results/'

In [ ]:
too.mkdirp(results_folder)    ## If directory does not exists, it is created here

In [ ]:
###Name of the file for the external input data, without the extension
datafile_ext = 'EXP_Pk_32_betas_60_redzs'
###Name of the file for the LCDM input data
datafile_ref = 'LCDM_Pk_60_redzs'

The following functions load the data into a object `emulation_data`. When the ratios are directly provide the user must indicate : `ratio_mode=True`

In [ ]:
emulation_data = dhl.DataHandle( datafile_ext, data_folder, datafile_ref, num_parameters=1) 
emulation_data.read_csv_pandas() 

## Calculate power spectra ratios at all redshifts available

Available **redshifts** are stored in the attribute `z_vals`

In [ ]:
# Available redshifts
emulation_data.z_vals

The function `calculate_ratio_by_redshifts` computes the ratio between the *extended* and *reference* model at each **redshit** passed as argument.

The user can optionally decide to **normalize** the data by passing `normalize=False`. This option will force all the ratios to be equal to 1 at k = `pos_norm`

In [ ]:
## Set normalize=False, since Fitting Formulae are already normalized
## First argument contains all the redshifts at which simulations are available
emulation_data.calculate_ratio_by_redshifts(emulation_data.z_vals,normalize=False)

# Define Parameters

Available parameters are stored in the attribute `emulation_data.extparam_vals`

In [ ]:
### Available parameters 
emulation_data.extparam_vals;

In [ ]:
emulation_data.max_train

In [ ]:
### Available parameters 
n_train = 30 # Number of training vectors without taking acount the extrema 

#  Define Classes (will be in a module)

In [ ]:
npca = 30

In [ ]:
filename='../interpolating_objects/CDEfittings-interpObj-new.sav'

In [ ]:
if not too.fileexists(filename):
    Interpolation = ito.Interpolating_function()

    for i,redshift in enumerate(emulation_data.z_requested):

        ratios_predicted , emulation_data,interpolation_function = dcl.Predict_ratio(emulation_data,Operator = "PCA",
                                                              train_noise = 1e-3, ##noise for the GP's kernel
                                                              gp_n_rsts = 10,##times allowed to restart the optimiser
                                                              ncomp=npca , ##number of components
                                                              gp_const = 1, ##Constant for the RBF kernel
                                                              gp_length = 1 , ## Length for  GP 
                                                              interp_type='GP', ##kind of interpolator,e.g int1d or GP 
                                                              n_splits = 1, ##number of splits
                                                              n_train=n_train, 
                                                               n_test=0,
                                                             train_redshift_indices = [i],
                                                             test_redshift_indices = [i],##indices of the test vectors
                                                             min_k =1e-2,max_k=10e1,return_interpolator=True)
        function = ito.Interpolating_function_redshift (emulation_data,interpolation_function,redshift,normalize=True)
        Interpolation.redshift_available.append(redshift)
        Interpolation.list_interpolation_function.append(function)
    with open(filename, 'wb') as f:
        joblib.dump(Interpolation, f)
else:
    print("File with interpolation object exists: ", filename)
    
print("Loading file from joblib")
with open(filename, 'rb') as f:
    Interpolation_loaded = joblib.load(f)
    

# Load and save the inperpolation function

In [ ]:
linkgrid=np.logspace(np.log10(0.012), np.log10(5.0), 100)
linkgrid;

In [ ]:
for bb in [0.05, 0.1, 0.12]:
    plt.semilogx(linkgrid,
             [Interpolation_loaded.predict(0.0,k,[bb]) for k in linkgrid ],
                label='beta='+str(bb))
    plt.semilogx(linkgrid,
             [Interpolation_loaded.predict(1.0,k,[bb]) for k in linkgrid ],
                ls='--', label='z=1.0, beta='+str(bb))
plt.legend()

In [ ]:
#plt.semilogx(np.power(10,emulation_data.masked_k_grid),emulation_data.matrix_datalearn_dict['theo']['train'])

#  Load cosmological packages

In [ ]:
import emcee

In [ ]:
import pyccl as ccl

In [ ]:
from multiprocessing import Pool

In [ ]:
Omega_m  = 0.32
Omega_b = 0.048
ns =0.95
bias =2.
h=0.70
As109=2.2
sigma8=0.8

In [ ]:
cosmo = ccl.Cosmology(Omega_c=Omega_m-Omega_b,
                       Omega_b=Omega_b,
                       h=h,
                       #A_s=As109*10**(-9),
                       sigma8=sigma8,
                       n_s=ns,
                       transfer_function='eisenstein_hu',
                       matter_power_spectrum='halofit')

In [ ]:
ccl.power.linear_matter_power(cosmo, 0.1, 1.0)

In [ ]:
linkgrid=np.logspace(np.log10(0.012), np.log10(5.0), 100)

In [ ]:
pklin_ccl=np.array([ccl.power.linear_matter_power(cosmo, kk, 1.0) for kk in linkgrid])
pknonlin_ccl=np.array([ccl.power.nonlin_matter_power(cosmo, kk, 1.0) for kk in linkgrid])

In [ ]:
plt.loglog(linkgrid,pklin_ccl)
plt.loglog(linkgrid,pknonlin_ccl)

In [ ]:
truth_beta=0.1

In [ ]:
def power_CDE(k_array, power_ref=None, extra_param=None, redshift=0.):
    Rofbeta = np.array([Interpolation_loaded.predict(redshift,k,[extra_param]) for k in k_array ])
    pnonlin_beta = Rofbeta*power_ref
    return pnonlin_beta

In [ ]:
plt.loglog(linkgrid,pklin_ccl, label='linear')
plt.loglog(linkgrid,pknonlin_ccl, label='non-linear')
plt.loglog(linkgrid, power_CDE(linkgrid,
                               power_ref=pknonlin_ccl,extra_param=0.1,
                               redshift=1.0),  label='non-linear-beta')
plt.legend()

In [ ]:
#plt.loglog(linkgrid,pklin_ccl, label='linear')
plt.semilogx(linkgrid,power_CDE(linkgrid,
                               power_ref=pknonlin_ccl,extra_param=0.1,
                               redshift=1.0)/pknonlin_ccl, label='beta enhancement')
#plt.loglog(linkgrid, pnonlin_beta, label='non-linear-beta')
plt.legend()

In [ ]:
def fiducial_power(cosmo_param):
    zfix=0.5
    afix = 1/(1+zfix)
    Omega_m, beta = cosmo_param
    Omega_b = 0.048
    ns =0.95
    bias =2.
    h=0.70
    As109=2.2
    sigma8=0.8
    
    kbins=np.logspace(np.log10(0.012), np.log10(5.0), 31)
    kspace=0.5*(kbins[1:] + kbins[:-1])
    
    cosmo = ccl.Cosmology(Omega_c=Omega_m-Omega_b,
                       Omega_b=Omega_b,
                       h=h,
                       #A_s=As109*10**(-9),
                       sigma8=sigma8,
                       n_s=ns,
                       transfer_function='eisenstein_hu',
                       matter_power_spectrum='halofit')
    
    
    pknonlin_ccl=np.array([ccl.power.nonlin_matter_power(cosmo, kk, afix) for kk in kspace])
    pknonlin_beta=power_CDE(kspace,power_ref=pknonlin_ccl,extra_param=beta,
                        redshift=zfix)
    
    return pknonlin_beta, kbins, kspace

In [ ]:
truth_beta

In [ ]:
truth_omegam=0.32

In [ ]:
pknlfid, kbins, kspace =  fiducial_power([truth_omegam,truth_beta])

In [ ]:
#plt.loglog(kspace,pknonlin_ccl, label='non-linear')
plt.loglog(kspace, pknlfid,  label='non-linear-beta obs truth')
plt.legend()

In [ ]:
np.savetxt("./results/pk_CDE_FittingFormula-observed-truth.txt", 
           np.column_stack([kspace,pknlfid]), 
           header='z=0.5,  beta=0.1, pk_nonlin=halofit, [k]=Mpc^-1')

In [ ]:
arr=np.loadtxt("./results/pk_CDE_FittingFormula-observed-truth.txt")
arr.shape

In [ ]:
def get_bounds():
    """
    Theoretical or numerical bounds on parameters
    """
    bounds = [
         (0.1, 0.5),     # Omega_m
         (0.05, 0.15)      # beta
    ]
    return np.array(bounds)

In [ ]:
def log_prior(cosmo_param):
    """ 
    Sets the prior functions
    """

    Omega_m, beta = cosmo_param

    # Get bounds
    bounds = get_bounds()

    if bounds[0][0] < Omega_m < bounds[0][1] and bounds[1][0] < beta < bounds[1][1]:
        return 0.0 
    return -np.inf

In [ ]:
def log_likelihood(cosmo_param, pk_obs, inv_cov):
    """
    defines the log likelihood
    """
    pknlfid, kbins, kspace =  fiducial_power(cosmo_param)
    
    x = pk_obs - pknlfid
    return -0.5* (x.T @ inv_cov @ x)

In [ ]:
def log_probability(cosmo_param, pk_obs, inv_cov):
    """
    """

    lp = log_prior(cosmo_param)

    if not np.isfinite(lp):
        return -np.inf
    
    return lp + log_likelihood(cosmo_param, pk_obs, inv_cov)

In [ ]:
def get_cov(cosmo_param):
    Volume = 50*(1000)**3
    ngal = 10**(-3)
    factor = 4*(np.pi**2)
    pnonlinfid, kbins, kspace = fiducial_power(cosmo_param)
    deltak=np.diff(kbins)
    cc1=Volume*kspace**2*deltak
    cc2=factor*(pnonlinfid+(1/ngal))**2
    cov=np.diag(cc2/cc1)
    return cov

In [ ]:
def run_emcee_mp(nsample=1000):
    """
    run MCMC for the parameters
    """
    param_dict ={}
   
    obs_vec = np.loadtxt("./results/pk_CDE_FittingFormula-observed-truth.txt")
    obs_vec=obs_vec[:,1]
    
    cov_simple = get_cov([0.32,0.1])
    print(np.linalg.cond(cov_simple))
    
    inv_cov = np.linalg.inv(cov_simple)
    previous_best = np.array([0.3, 0.09])
    
    ndim = len(previous_best)

    pos = previous_best + 1e-3 * np.random.randn(16, ndim)
    nwalkers, ndim = pos.shape

    #with Pool() as pool:
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(obs_vec, inv_cov)) #, pool=pool)
    sampler.run_mcmc(pos, nsample, progress=True)

    return sampler

In [ ]:
sample = run_emcee_mp(1000)

In [ ]:
fig, axes = plt.subplots(2, figsize=(10, 7), sharex=True)
samples = sample.get_chain()
labels = ["Om", "beta"]
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
#sample = run_emcee_mp(100)